Multilayer Perceptron(MLP) on Iris Dataset

In [2]:
%pip install tensorflow

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [33]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from tensorflow.keras import Input
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [ ]:
#Load datasets
df = pd.read_csv("bmi.csv")
X = df[["Age", "Height", "Weight", "Bmi"]].values
Y = df["BmiClass"].values
print(df.head())

   Age  Height  Weight        Bmi       BmiClass
0   61    1.85  109.30  31.935720  Obese Class 1
1   60    1.71   79.02  27.023700     Overweight
2   60    1.55   74.70  31.092612  Obese Class 1
3   60    1.46   35.90  16.841809    Underweight
4   60    1.58   97.10  38.896010  Obese Class 2


In [ ]:
le = LabelEncoder()
Y_encoded = le.fit_transform(Y)

print("Class labels: ", le.classes_) #Labels
print("Encoded y: ", Y_encoded[:5]) #Numeric targets

Class labels:  ['Normal Weight' 'Obese Class 1' 'Obese Class 2' 'Obese Class 3'
 'Overweight' 'Underweight']
Encoded y:  [1 4 1 5 2]


In [ ]:
#Converting BMI data to a pandas DataFrame
bmi_df = pd.DataFrame(X, columns=["Age", "Height", "Weight", "Bmi"])

#Adding the target varibale in the DataFrame
bmi_df['target'] = Y_encoded

#display first 5 rows
display(bmi_df.head())

,Age,Height,Weight,Bmi,target
0,61.0,1.85,109.30,31.935720,1
1,60.0,1.71,79.02,27.023700,4
2,60.0,1.55,74.70,31.092612,1
3,60.0,1.46,35.90,16.841809,5
4,60.0,1.58,97.10,38.896010,2


In [15]:
bmi_df.describe()

,Age,Height,Weight,Bmi,target
count,741.000000,741.000000,741.000000,741.000000,741.000000
mean,31.618084,1.709427,78.412497,26.365427,1.970310
std,11.655466,0.085974,32.254547,9.223191,2.005852
min,15.000000,1.460000,25.900000,12.150497,0.000000
25%,22.000000,1.670000,63.000000,22.129740,0.000000
50%,29.000000,1.721000,72.900000,24.132412,2.000000
75%,40.000000,1.751000,83.300000,27.249306,4.000000
max,61.000000,2.070000,270.000000,66.301350,5.000000


In [17]:
x = bmi_df.drop('target', axis=1)
y = bmi_df['target']
bmi_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 741 entries, 0 to 740
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Age     741 non-null    float64
 1   Height  741 non-null    float64
 2   Weight  741 non-null    float64
 3   Bmi     741 non-null    float64
 4   target  741 non-null    int64  
dtypes: float64(4), int64(1)
memory usage: 29.1 KB


In [19]:
#Prepocessor
scaler = StandardScaler()
x_scaled = scaler.fit_transform(x)
y_encoded = to_categorical(y)
x_scaled.shape, y_encoded.shape

((741, 4), (741, 6))

In [21]:
X_train, X_test, y_train, y_test = train_test_split(x_scaled, y_encoded, test_size=0.2, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((592, 4), (149, 4), (592, 6), (149, 6))

In [ ]:
#Building MLP Model
model = Sequential([Input(shape=(4,)), Dense(32, activation='relu'), Dense(16, activation='relu'), Dense(6, activation='softmax')])

#Compilation of the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

chkpt = ModelCheckpoint(
    "best_model.h5",
    monitor="val_accuracy",
    save_best_only=True,
    mode="max",
    verbose=0
)

es = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
#Training of the Model
history = model.fit(X_train, y_train, epochs=200, validation_data=(X_test, y_test), callbacks=[chkpt, es], verbose=0)

#Evaluation of the model's accuracy
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

Epoch 1/200
 1/19 ━━━━━━━━━━━━━━━━━━━━ 9s 555ms/step - accuracy: 0.2188 - loss: 1.9715

19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.2145 - loss: 1.8810 - val_accuracy: 0.3289 - val_loss: 1.7164
Epoch 2/200
 1/19 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.3438 - loss: 1.7026

19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.3378 - loss: 1.7182 - val_accuracy: 0.4832 - val_loss: 1.6094
Epoch 3/200
 1/19 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.4062 - loss: 1.5858

19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5439 - loss: 1.6020 - val_accuracy: 0.6040 - val_loss: 1.5206
Epoch 4/200
 1/19 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - accuracy: 0.5938 - loss: 1.4692

19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6453 - loss: 1.4895 - val_accuracy: 0.6913 - val_loss: 1.4088
Epoch 5/200
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6824 - loss: 1.3570 - val_accuracy: 0.6913 - val_loss: 1.2882
Epoch 6/200
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6875 - loss: 1.2233 - val_accuracy: 0.6779 - val_loss: 1.1615
Epoch 7/200
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7027 - loss: 1.0843 - val_accuracy: 0.6913 - val_loss: 1.0365
Epoch 8/200
 1/19 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.5938 - loss: 1.0686

...

19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9595 - loss: 0.1410 - val_accuracy: 0.9664 - val_loss: 0.1620
Epoch 55/200
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9578 - loss: 0.1375 - val_accuracy: 0.9597 - val_loss: 0.1581
Epoch 56/200
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9645 - loss: 0.1352 - val_accuracy: 0.9597 - val_loss: 0.1540
Epoch 57/200
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9595 - loss: 0.1324 - val_accuracy: 0.9664 - val_loss: 0.1516
Epoch 58/200
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9628 - loss: 0.1301 - val_accuracy: 0.9664 - val_loss: 0.1456
Epoch 59/200
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9628 - loss: 0.1264 - val_accuracy: 0.9664 - val_loss: 0.1429
Epoch 60/200
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9645 - loss: 0.1242 - val_accuracy: 0.9664 - val_loss: 0.1397
Epoch 61/200
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9628 - loss: 0.1216 - val_accuracy: 0.9664 - val_lo

19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9713 - loss: 0.1150 - val_accuracy: 0.9732 - val_loss: 0.1251
Epoch 66/200
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9628 - loss: 0.1122 - val_accuracy: 0.9664 - val_loss: 0.1228
Epoch 67/200
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9679 - loss: 0.1104 - val_accuracy: 0.9732 - val_loss: 0.1208
Epoch 68/200
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9747 - loss: 0.1089 - val_accuracy: 0.9732 - val_loss: 0.1175
Epoch 69/200
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9679 - loss: 0.1076 - val_accuracy: 0.9732 - val_loss: 0.1158
Epoch 70/200
 1/19 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.9688 - loss: 0.1095

19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9730 - loss: 0.1046 - val_accuracy: 0.9799 - val_loss: 0.1143
Epoch 71/200
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9747 - loss: 0.1035 - val_accuracy: 0.9799 - val_loss: 0.1112
Epoch 72/200
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9730 - loss: 0.1015 - val_accuracy: 0.9799 - val_loss: 0.1100
Epoch 73/200
 1/19 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.9688 - loss: 0.1093

19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9730 - loss: 0.1009 - val_accuracy: 0.9866 - val_loss: 0.1075
Epoch 74/200
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9730 - loss: 0.0992 - val_accuracy: 0.9866 - val_loss: 0.1075
Epoch 75/200
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9730 - loss: 0.0973 - val_accuracy: 0.9799 - val_loss: 0.1033
Epoch 76/200
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9747 - loss: 0.0979 - val_accuracy: 0.9866 - val_loss: 0.1023
Epoch 77/200
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9780 - loss: 0.0954 - val_accuracy: 0.9799 - val_loss: 0.1013
Epoch 78/200
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9764 - loss: 0.0939 - val_accuracy: 0.9799 - val_loss: 0.1001
Epoch 79/200
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9713 - loss: 0.0927 - val_accuracy: 0.9799 - val_loss: 0.0976
Epoch 80/200
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9730 - loss: 0.0903 - val_accuracy: 0.9799 - val_lo

19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9916 - loss: 0.0463 - val_accuracy: 0.9933 - val_loss: 0.0569
Epoch 146/200
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9932 - loss: 0.0461 - val_accuracy: 0.9866 - val_loss: 0.0568
Epoch 147/200
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9916 - loss: 0.0459 - val_accuracy: 0.9866 - val_loss: 0.0538
Epoch 148/200
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9949 - loss: 0.0449 - val_accuracy: 0.9866 - val_loss: 0.0553
Epoch 149/200
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9916 - loss: 0.0439 - val_accuracy: 0.9866 - val_loss: 0.0539
Epoch 150/200
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9916 - loss: 0.0431 - val_accuracy: 0.9866 - val_loss: 0.0554
Epoch 151/200
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9949 - loss: 0.0427 - val_accuracy: 0.9933 - val_loss: 0.0528
Epoch 152/200
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9966 - loss: 0.0428 - val_accuracy: 0.9866 -

In [34]:
best = load_model("best_model.h5")
loss, acc = best.evaluate(X_test, y_test, verbose=0)
print(f"Best model test accuracy: {acc*100:.2f}%")

Best model test accuracy: 99.33%
